## Tải yolov5, import thư viện và cuda

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output # to display images
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16114, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 16114 (delta 8), reused 10 (delta 1), pack-reused 16089
Receiving objects: 100% (16114/16114), 14.85 MiB | 23.11 MiB/s, done.
Resolving deltas: 100% (11031/11031), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Truy cập tới API cá nhân của Roboflow

In [ ]:
api_key = "nem4y7IagBKhlPKqH0yb"
from roboflow import Roboflow
rf = Roboflow(api_key=api_key, model_format="yolov5", notebook="ultralytics")

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

## Truy cập tới dataset của nhóm trên Roboflow

In [ ]:
project = rf.workspace("trng-tp").project("vietnamtrafficsign-6cul5")
dataset = project.version(5).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /content/datasets/VietnamTrafficSign-5 in yolov5pytorch:: 100%|██████████| 17342/17342 [00:04<00:00, 3990.31it/s]


## Train mô hình với 100 epochs

In [ ]:
!python train.py --img 416 --batch 64 --epochs 100 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache --workers 2

2023-12-26 08:53:39.013975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-26 08:53:39.014030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-26 08:53:39.016026: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/datasets/VietnamTrafficSign-5/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=64, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, 

## Vẽ sơ đồ thống kê bằng TensorBoard
(sơ đồ thống kê có thể được tìm thấy ở địa chỉ `yolov5/runs/train`)

In [ ]:
#Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

Launching TensorBoard...

## Test ảnh trong tập test sử dụng file weight `best.pt` vừa mới train

In [ ]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.2 --source {dataset.location}/valid/images

## Hiện 20 ảnh vừa mới test

In [ ]:
#display inference on ALL test images
import glob
from IPython.display import Image, display
i = 0
# Choose the correct exp folder - see prev output block
for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
  i += 1
  if i < 20:
    display (Image(filename=imageName))
    print("\n")

## Test video từ bên ngoài sử dụng file weight `best.pt` vừa mới train

In [ ]:
!python detect.py --weights '/content/best.pt' --img 416 --conf 0.2 --source '/content/video2.mp4'